In [ ]:
from src.db.db_utils import db_utils
from src.db.controller import init_async_db
from src.config import ETHER_protocols
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

In [ ]:

transactions, blocks, future = await init_async_db(True)
db_utils.set_tables(transactions, blocks, future)

transaction_table = db_utils.get_transactions()

protocol = ETHER_protocols.get('RoninBridge').lower()
transactions_rows = await transaction_table.get_all_rows_by_criteria({'contract': protocol})
print(len(transactions_rows))


In [ ]:
print(protocol)

In [ ]:

df = pd.DataFrame([t.__dict__ for t in transactions_rows])

In [ ]:
df.head()

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [ ]:
df = df.set_index('timestamp').resample('H').max().reset_index()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
fig = px.line(df.reset_index(), x='timestamp', y='priority_fee', title='NYC Taxi Demand')
#slider
fig.update_xaxes(
    rangeslider_visible = True,
    rangeselector = dict(
        buttons = list([
              dict(count=1, label='1y', step="year", stepmode="backward"),
              dict(count=2, label='2y', step="year", stepmode="backward"),
              dict(count=2, label='5y', step="year", stepmode="backward")
        ])
    )
)
fig.show()

In [ ]:
from prophet import Prophet
taxi_df = df.reset_index()[['timestamp', 'priority_fee']].rename({'timestamp':'ds', 'priority_fee':'y'}, axis='columns')

In [ ]:
train = taxi_df
# test = taxi_df[(taxi_df['ds'] > '2022-03-28')]

In [ ]:
train['cap'] = 9 * 10**13
train['floor'] = 0

m = Prophet(growth='logistic')
m.fit(train)

In [ ]:
future = m.make_future_dataframe(periods=24, freq='H')
future['cap'] = 10**13
future['floor'] = 0

In [ ]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
forecast.shape

In [ ]:
result = pd.concat([taxi_df.set_index('ds')['y'], forecast.set_index('ds')[['yhat','yhat_lower','yhat_upper']]], axis=1)
fig1 = m.plot(forecast)

In [ ]:
result['error'] = result['y'] - result['yhat']
result['uncertainty'] = result['yhat_upper'] - result['yhat_lower']

In [ ]:
result['anomaly'] = result.apply(lambda x: 'Yes' if(np.abs(x['error']) > 1.5*x['uncertainty']) else 'No', axis = 1)

In [ ]:
fig = px.scatter(result.reset_index(), x='ds', y='y', color='anomaly', title='NYC Taxi Demand')
# fig.add_scatter(result, x='ds', y='yhat', color='green', title='real')
#slider
fig.update_xaxes(
    rangeslider_visible = True,
    rangeselector = dict(
        buttons = list([
              dict(count=1, label='1y', step="year", stepmode="backward"),
              dict(count=2, label='3y', step="year", stepmode="backward"),
              dict(count=2, label='5y', step="year", stepmode="backward"),
              dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
for index, row in forecast.iterrows():
    print({'contract': protocol, 'timestamp': row['ds'].timestamp(), 'priority_fee': row['yhat'],
                            'priority_fee_lower': row['yhat_lower'], 'priority_fee_upper': row['yhat_upper']})